In [3]:
from scipy.fftpack import fft
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
import glob
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn import svm, tree

import numpy as np
import matplotlib.pyplot as plt
import csv
from merge_script import modify_first_column
import math
from sklearn.decomposition import PCA
import glob
import Calculate_Feature
import os
from spline import np_move_avg
import pandas as pd

def use_pca(featuer_data, n_components=5):
    pca = PCA(n_components)
    axis_fea = pca.fit_transform(featuer_data)  # 每个样本降为n_components维
    fea = []
    for raw in range(axis_fea.shape[0]):
        for ele in axis_fea[raw, :]:
            fea.append(ele)
    return fea


def axis_normalization(datalist, stage=1):
    normalized_data = []
    #datalist = np.array(datalist)
    for data in datalist:
        maximum = max(data)
        minimum = min(data)
        # print("maximum is:",maximum,"minimum is:",minimum)
        normalized_data.append([(item - minimum) / (maximum - minimum) * stage for item in data])
    return np.array(normalized_data)


def three_fuse(data):
    axis_num = int(data.shape[0] / 3)
    f_nd = []

    for f_in in range(axis_num):
        nd = []
        index = f_in * 3
        for i in range(data.shape[1]):
            d = data[:, i]
            fuse_data = math.sqrt(d[index] * d[index] + d[index + 1] * d[index + 1] + d[index + 2] * d[index + 2])
            nd.append(fuse_data)
        f_nd.append(nd)
    return f_nd

In [55]:
scale_list = ["0.5", "0.7", "0.9", "1.1", "1.3", "1.5"]
# scale_list = ["0.7", "0.9", "1.1", "1.3", "1.5"]
scale_list_conv = ["0", "1", "2", "3"]
human_name = ["Effy", "Leafy", "Nick", "Qin", "Tonii", "Xu", "Yamamoto"]

virtual_total_file = []
real_total_file = []
total_file = []
virtual_data_path_3 = "save_data/Pos_New_New_VirtualData_addPelvis_xyz_25_spline3_240/"
virtual_data_path_4 = "save_data/Virtual_New_Conv_notrans_spline3_240/"
Real_data_path = "save_data/RealData_Seg_byName/"

for scale in scale_list:
    virtual_total_file.append(virtual_data_path_3 + scale)
    total_file.append(virtual_data_path_3 + scale)
#
# for scale in scale_list_conv:
#     virtual_total_file.append(virtual_data_path_4 + scale)
#     total_file.append(virtual_data_path_4 + scale)

for name in human_name:
    real_total_file.append(Real_data_path + name)
    total_file.append(Real_data_path + name)


print(total_file)

['save_data/Pos_New_New_VirtualData_addPelvis_xyz_25_spline3_240/0.5', 'save_data/Pos_New_New_VirtualData_addPelvis_xyz_25_spline3_240/0.7', 'save_data/Pos_New_New_VirtualData_addPelvis_xyz_25_spline3_240/0.9', 'save_data/Pos_New_New_VirtualData_addPelvis_xyz_25_spline3_240/1.1', 'save_data/Pos_New_New_VirtualData_addPelvis_xyz_25_spline3_240/1.3', 'save_data/Pos_New_New_VirtualData_addPelvis_xyz_25_spline3_240/1.5', 'save_data/RealData_Seg_byName/Effy', 'save_data/RealData_Seg_byName/Leafy', 'save_data/RealData_Seg_byName/Nick', 'save_data/RealData_Seg_byName/Qin', 'save_data/RealData_Seg_byName/Tonii', 'save_data/RealData_Seg_byName/Xu', 'save_data/RealData_Seg_byName/Yamamoto']


In [56]:
label = [0, 1, 2, 3, 4, 5, 6, 7]
LABEL_NUM = len(label)
encode_feauture = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]  #[1,1,1,1,1,1,1,1,1,1,1,1]
# encode_feauture = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]  #[1,1,1,1,1,1,1,1,1,1,1,1]
Raw_Data = []
Data_X = []
Label_X = []
PCA_Data = []

# save_data/ori/pos_data/ankleTap_o_pos_merge_data.csv

virtual_motion_type = ['ankle', 'highKnee', 'Knee_kick', 'reverseLunge', 'sideCrunch', 'sidetoside', 'warm']
for fold_path in total_file:
    for i in range(len(virtual_motion_type)):
        motion = virtual_motion_type[i]
        motion_file = glob.glob(os.path.join(fold_path + '/' + '*' + motion + '*', '*.csv'))
        # print(motion_file)
        for motion_frame in motion_file:
            # print(motion_frame)

            df = pd.read_csv(motion_frame, index_col=False, header=None, usecols= [9,10,11])
            df = np.array(df).T
            # print(df.shape[0])

            # for k in range(df.shape[0]):
            #     #df[:,k] = signal.filtfilt(b,a,df[:,k])
            #     df[k, :] = np_move_avg(df[k, :], 30, mode='same')

            fuse_df = three_fuse(df)
            # plt.plot(fuse_df[0])
            # pca = PCA(1)
            # ld_signal = pca.fit_transform(df.T)
            # print(ld_signal.shape)
            # pca_signal = []
            # for d in ld_signal:
            #     pca_signal.append(float(d))
            # pca_nor_data = []
            # maximum = max(pca_signal)
            # minimum = min(pca_signal)
            # for item in pca_signal:
            #     pca_nor_data.append((item - minimum) / (maximum - minimum) * 1)
            nor_df = axis_normalization(fuse_df)  #.tolist()
            # plt.plot(nor_df)
            # print(nor_df.shape)
            axis_fea = []
            rd = []
            # print(nor_df.shape[0])
            for raw in range(nor_df.shape[0]):
                tmd = nor_df[raw, :]
                # print(tmd.shape)
                rd.append(tmd)
                cal_fea = Calculate_Feature.Get_Feature(tmd, encode_feauture)
                fea = cal_fea.cal_result()
                # print(len(fea))
                #axis_fea.append(fea)
                # print(len(axis_fea))
                for f in fea:
                    axis_fea.append(f)
            # print(len(axis_fea))
            Data_X.append(axis_fea)
            Label_X.append(label[i])
            Raw_Data.append(rd[0])
            # PCA_Data.append(pca_nor_data)

Virtual_Data = np.array(Data_X)
print(Virtual_Data.shape)
# print(Real_Data[0])
# for i in range(Virtual_Data.shape[0]):
#     Virtual_Data[i] = [0 if math.isnan(x) else x for x in Virtual_Data[i]]
# plt.plot(Real_Data[])
# print(Real_Data.shape)
Row_Virtual_Data = np.array(Raw_Data)
Raw_PCA_Data = np.array(PCA_Data)
#pca = PCA(n_components = 2)
#Real_Data = pca.fit_transform(Real_Data)
Virtual_Label = np.array(Label_X)




(1552, 20)


In [34]:
def k_fold_cv(data,lab,n_fold = 5):
    #n_fold = 3
    k_fold = KFold(n_splits = n_fold, shuffle=True)
    tmp_train, rf_test, svm_test, svm2_test = list(), list(), list(), list()
    conf_mat = np.zeros((LABEL_NUM,LABEL_NUM))
    #print(Data.shape)
    rf = RandomForestClassifier(n_estimators = 30, random_state=42)
    svc = svm.SVC(C=1000, kernel='rbf')
    # clf = TimeSeriesSVC(C=5.4, kernel="gak", gamma="auto", max_iter=-1, n_jobs=-1, decision_function_shape='ovr')

    index = 0
    for train, test in k_fold.split(data):
        #print(train)
        index += 1
        tmp_trx = []
        tmp_try = []
        tmp_raw = []
        for titer in range(len(train)):
            tmp_trx.append(data[train[titer]])
            #tmp_raw.append(data2[train[titer]])
            tmp_try.append(lab[train[titer]])
        #print(tmp_trx)
        rf.fit(tmp_trx, tmp_try)
        svc.fit(tmp_trx, tmp_try)

        #clf.fit(tmp_raw, tmp_try)

        tmp_tex = []
        tmp_te_raw_x = []
        tmp_tey = []
        for titer in range(len(test)):
            tmp_tex.append(data[test[titer]])
            #tmp_te_raw_x.append(data2[test[titer]])
            tmp_tey.append(lab[test[titer]])

            rf_predicted = rf.predict(tmp_tex)
            svc_predicted = svc.predict(tmp_tex)
            #svc2_predicted = clf.predict(tmp_te_raw_x)

        rf_test.append(accuracy_score(tmp_tey,rf_predicted))
        svm_test.append(accuracy_score(tmp_tey,svc_predicted))
        #svm2_test.append(accuracy_score(tmp_tey,svc2_predicted))
        #tmp_test.append(svc.score(tmp_tex, tmp_tey))
        #print("result of svm classifier", accuracy_score(tmp_tey,svc_predicted))
        #print("result of random forest classifier", accuracy_score(tmp_tey,rf_predicted))

        # confusion matrix calculation
        #print(confusion_matrix(tmp_tey,rf_predicted))
        #conf_mat += confusion_matrix(tmp_tey,rf_predicted)

    rf_score = sum(rf_test) / len(rf_test)
    svm_score = sum(svm_test) / len(svm_test)
    #svm2_score = sum(svm2_test) / len(svm2_test)

    return svm_score,rf_score



In [53]:
pca = PCA(2)
Virtual_Data_PCA = pca.fit_transform(Virtual_Data)
Virtual_Data_PCA.shape

(1985, 2)

0.8116980023501764 0.8755522914218566

In [58]:
svm_result, rf_result = k_fold_cv(Virtual_Data, Virtual_Label,5)  # Virtual_Data_PCA
print(svm_result, rf_result)

0.8286132143968468 0.9065677834249559


0.609294947121034 0.5497708578143361

In [50]:
svm_result, rf_result = k_fold_cv(Virtual_Data_PCA, Virtual_Label,5)  # Virtual_Data_PCA
print(svm_result, rf_result)

0.49741154764488815 0.4613115910570871
